# 切片

切片（Slice）是一个拥有相同元素的可变长度的序列。它是基于数组类型做的一层封装。非常灵活，支持自动扩容。

切片是一个引用类型，它内部结构包含地址、长度和容量。切片一般用于快速的操作一块数据集合。

切片的容量是从它的第一个元素开始数，到其底层数组元素末尾的个数。

## 定义
声明切片类型的基本语法：
```
var name []T
```
- name：表示变量名
- T：表示切片中的元素类型

In [20]:
%%
// 声明切片类型
var a []string              //声明一个字符串切片
var b = []int{}             //声明一个整型切片并初始化
var c = []bool{false, true} //声明一个布尔切片并初始化
// var d = []bool{false, true} //声明一个布尔切片并初始化
fmt.Println(a)              //[]
fmt.Println(b)              //[]
fmt.Println(c)              //[false true]
fmt.Println(a == nil)       //true
fmt.Println(b == nil)       //false
fmt.Println(c == nil)       //false
// fmt.Println(c == d)   //切片是引用类型，不支持直接比较，只能和nil比较

[]
[]
[false true]
true
false
false


### 基于数组来创建的切片
#### 简单切片表达式
切片的底层就是一个数组，所以我们可以基于数组通过切片表达式得到切片。切片表达式中的low和hight表示一个索引范围（左包含，右不包含），也就是下面代码中从数组a中选出1<=索引值<4的元素组成切片s，得到的切片长度=high-low，容量等于得到的切片的底层数组的容量。

In [21]:
%%
a := [5]int{1, 2, 3, 4, 5}
s := a[1:3]  // s := a[low:high]
fmt.Printf("s:%v len(s):%v cap(s):%v\n", s, len(s), cap(s))

s:[2 3] len(s):2 cap(s):4


#### 完整切片表达式
对于数组，指向数组的指针，或切片a(注意不能是字符串)支持完整切片表达式：
```
a[low : high : max]
```
上面的代码会构造与简单切片表达式`a[low: high]`想、相同类型、相同长度和元素的切片。另外，它会将得到的结果切片的容量设置为`max-low`。在完整切片表达式中只有第一个索引值（low）可以省略；它默认为0。

In [22]:
%%
a := [5]int{1, 2, 3, 4, 5}
t := a[1:3:5]
fmt.Printf("t:%v len(t):%v cap(t):%v\n", t, len(t), cap(t))

t:[2 3] len(t):2 cap(t):4


完整切片表达式需要满足的条件是`0 <= low <= high <= max <= cap(a)`，其他条件和简单切片表达式相同。

### 使用make()函数构造切片
我们上面都是基于数组来创建的切片，如果需要动态创建一个切片，我们就需要使用内置的make()函数，格式如下：
```
make([]T, size, cap)
```
- T：切片的元素类型
- size：切片中元素的数量
- cap：切片的容量

In [23]:
%%
a := make([]int, 2, 10)
fmt.Println(a)      //[0 0]
fmt.Println(len(a)) //2
fmt.Println(cap(a)) //10

[0 0]
2
10


切片的本质
> 切片的本质就是随底层数组的封装，它包含了三个信息：底层数组的指针、切片的长度（len）和切片的容量（cap）。

判断切片是否为空
> 要检查切片是否为空，需要使用len(s)==0来判断，不要使用s == nil来判断。

### 切片不能直接比较
切片之间是不能比较的，我们不能使用==操作符来判断两个切片是否含有全部相等元素。切片唯一合法的比较是和nil比较。一个nil值的切片并没有底层数组，一个nil值的切片的长度和容量都是0。但是不能说一个长度和容量都是0的切片一定是nil，因为切片初始化为空值的时候的长度和容量也都是0，但不等于nil

In [24]:
%%
var s1 []int
s2 := []int{}
s3 := make([]int, 0)
fmt.Println(len(s1), cap(s1), s1==nil)
fmt.Println(len(s2), cap(s2), s2!=nil)
fmt.Println(len(s3), cap(s3), s3!=nil)


0 0 true
0 0 true
0 0 true


所以判断切片是否为空，要用len(s)==0来判断，不应该使用s == nil来判断

### 切片的赋值拷贝
拷贝前后两个变量共享底层数组，对一个切片的修改会影响零一个切片的内容，需要注意。

函数外修改切片会影响切片本身

In [25]:
%%
s1 := make([]int, 3) //[0 0 0]
s2 := s1             //将s1直接赋值给s2，s1和s2共用一个底层数组
s2[0] = 100
fmt.Println(s1) //[100 0 0]
fmt.Println(s2) //[100 0 0]

[100 0 0]
[100 0 0]


### 切片遍历
切片的遍历方式和数组一样，支持索引比哪里和for range遍历。

In [26]:
%%
s := []int{1, 3, 5}

for i := 0; i < len(s); i++ {
	fmt.Println(i, s[i])
}

for index, value := range s {
	fmt.Println(index, value)
}

0 1
1 3
2 5
0 1
1 3
2 5


### append()方法为切片添加元素
Go语言的内建函数append()可以为切片动态添加元素。可以一次添加一个元素，可以添加多个元素，也可以添加另一个切片中的元素（否面加...）。

In [27]:
%%
var s []int
s = append(s, 1)        // [1]
s = append(s, 2, 3, 4)  // [1 2 3 4]
s2 := []int{5, 6, 7}  
s = append(s, s2...)
fmt.Println(s)
fmt.Println(s2)

[1 2 3 4 5 6 7]
[5 6 7]


注意：通过var声明的零值切片可以在append()函数直接使用，无需初始化。

In [28]:
%%
var s []int
s = append(s, 1, 2, 3)
fmt.Println(s)

[1 2 3]


### 使用copy()函数赋值切片
由于切片是引用类型，所以a和b其实都指向了同一块内存地址。修改b的同时a的值也会发生变化。

Go语言内建的copy函数可以迅速地将一个切片的数据复制到另一个切片空间中，copy()函数的使用格式：
```
copy(destSlice, srcSlice []T)
```
总结：
- srcSlice：数据来源切片
- destSlice：目标切片

In [29]:
%%
// copy()复制切片
a := []int{1, 2, 3, 4, 5}
c := make([]int, 5, 5)
copy(c, a)     //使用copy()函数将切片a中的元素复制到切片c
fmt.Println(a) //[1 2 3 4 5]
fmt.Println(c) //[1 2 3 4 5]
c[0] = 1000
fmt.Println(a) //[1 2 3 4 5]
fmt.Println(c) //[1000 2 3 4 5]

[1 2 3 4 5]
[1 2 3 4 5]
[1 2 3 4 5]
[1000 2 3 4 5]


### 从切片中删除元素
Go语言中并没有删除切片元素的专用方法，我们可以使用切片本身的特性来删除元素。

In [30]:
%%
// 从切片中删除元素
a := []int{30, 31, 32, 33, 34, 35, 36, 37}
// 要删除索引为2的元素
a = append(a[:2], a[3:]...)
fmt.Println(a)

[30 31 33 34 35 36 37]


删除元素：从切片a中删除索引为`index`的元素，操作方法是`a=append(a[:index], a[index+1:]...)`